<a href="https://colab.research.google.com/github/profepedrorojas/Computer-Science-Resources/blob/master/ProyectoFinal_Visualizacion_TSLA_GME_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Final - Visualización de Acciones e Ingresos (TSLA y GME)

In [1]:
!pip install yfinance bs4 lxml plotly --quiet

In [2]:

import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots


## Pregunta 1: Extraer datos de acciones de Tesla usando yfinance

In [3]:

tesla = yf.Ticker("TSLA")
tesla_share_price_data = tesla.history(period="5y")
tesla_share_price_data.reset_index(inplace=True)
tesla_share_price_data.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2020-04-22 00:00:00-04:00,46.931999,48.933334,45.914001,48.807335,212482500,0.0,0.0
1,2020-04-23 00:00:00-04:00,48.506668,48.933334,46.875332,47.042000,198550500,0.0,0.0
2,2020-04-24 00:00:00-04:00,47.387333,48.715332,46.545334,48.343334,198180000,0.0,0.0
3,2020-04-27 00:00:00-04:00,49.174000,53.299332,49.000000,53.250000,310221000,0.0,0.0
4,2020-04-28 00:00:00-04:00,53.042667,53.666668,50.445999,51.274666,228330000,0.0,0.0


## Pregunta 2: Extraer ingresos de Tesla con web scraping

In [5]:

# URL de Macrotrends para Tesla
url_tesla = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"

# Obtener HTML
html_data = requests.get(url_tesla, headers={"User-Agent": "Mozilla/5.0"}).text

# Usar BeautifulSoup para buscar todas las tablas
soup = BeautifulSoup(html_data, "html.parser")
tables = soup.find_all("table")

# Filtrar la tabla adecuada por inspección visual (normalmente la primera útil es la [1])
df_list = pd.read_html(str(tables))
for df in df_list:
    if "Revenue" in df.columns[1]:
        tesla_revenue = df
        break

# Limpiar datos
tesla_revenue.columns = ['Date', 'Revenue']
tesla_revenue['Revenue'] = tesla_revenue['Revenue'].str.replace(r'[$,]', '', regex=True)
tesla_revenue['Revenue'] = pd.to_numeric(tesla_revenue['Revenue'], errors='coerce')
tesla_revenue.dropna(inplace=True)
tesla_revenue.tail()


<ipython-input-5-052f775404b1>:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_list = pd.read_html(str(tables))


,Date,Revenue
11,2013,2013
12,2012,413
13,2011,204
14,2010,117
15,2009,112


## Pregunta 3: Extraer datos de acciones de GameStop usando yfinance

In [6]:

gme = yf.Ticker("GME")
gme_share_price_data = gme.history(period="5y")
gme_share_price_data.reset_index(inplace=True)
gme_share_price_data.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2020-04-22 00:00:00-04:00,1.2850,1.2925,1.2075,1.2225,10711200,0.0,0.0
1,2020-04-23 00:00:00-04:00,1.1875,1.2225,1.1450,1.1750,9063600,0.0,0.0
2,2020-04-24 00:00:00-04:00,1.1950,1.2125,1.1650,1.1925,8944800,0.0,0.0
3,2020-04-27 00:00:00-04:00,1.2125,1.4975,1.2025,1.4550,29100400,0.0,0.0
4,2020-04-28 00:00:00-04:00,1.4800,1.5100,1.2650,1.4100,20800800,0.0,0.0


## Pregunta 4: Extraer ingresos de GameStop con web scraping

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url_gme = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"
html_data = requests.get(url_gme, headers={"User-Agent": "Mozilla/5.0"}).text

soup = BeautifulSoup(html_data, "html.parser")
tables = soup.find_all("table")

# Leer todas las tablas y filtrar la que contiene la columna 'Revenue'
df_list = pd.read_html(str(tables))
for df in df_list:
    if "Revenue" in df.columns[1]:
        gme_revenue = df
        break

# Normalizar columnas y limpiar datos
gme_revenue.columns = ['Date', 'Revenue']
gme_revenue['Revenue'] = gme_revenue['Revenue'].str.replace(r'[$,]', '', regex=True)
gme_revenue['Revenue'] = pd.to_numeric(gme_revenue['Revenue'], errors='coerce')
gme_revenue.dropna(inplace=True)

# Mostrar resultado
gme_revenue.tail()


<ipython-input-8-47f4e3134708>:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_list = pd.read_html(str(tables))


,Date,Revenue
12,2013,8887
13,2012,9551
14,2011,9474
15,2010,9078
16,2009,8806


## Pregunta 5: Gráfico de acciones e ingresos de Tesla

In [9]:

fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    subplot_titles=("Precio de la acción de Tesla", "Ingresos de Tesla"),
                    vertical_spacing=0.3)
fig.add_trace(go.Scatter(x=tesla_share_price_data['Date'], y=tesla_share_price_data['Close'], name="Precio TSLA"), row=1, col=1)
fig.add_trace(go.Bar(x=tesla_revenue['Date'], y=tesla_revenue['Revenue'], name="Ingresos TSLA"), row=2, col=1)
fig.update_layout(height=600, title_text="Tesla - Precio de la Acción e Ingresos")
fig.show()


## Pregunta 6: Gráfico de acciones e ingresos de GameStop

In [10]:

fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    subplot_titles=("Precio de la acción de GME", "Ingresos de GME"),
                    vertical_spacing=0.3)
fig.add_trace(go.Scatter(x=gme_share_price_data['Date'], y=gme_share_price_data['Close'], name="Precio GME"), row=1, col=1)
fig.add_trace(go.Bar(x=gme_revenue['Date'], y=gme_revenue['Revenue'], name="Ingresos GME"), row=2, col=1)
fig.update_layout(height=600, title_text="GameStop - Precio de la Acción e Ingresos")
fig.show()


## Pregunta 7: Compartir cuaderno

Puedes subir este notebook a GitHub y compartir el enlace con tu instructor o compañeros.